# 페이퍼의 이름에 저널이 포함되어 있는 경우를 다시 나눠주는 튜닝작업

#### 튜닝 작업에는 기존에 저널 카테고리에 있던 저널 이름들을 일반화시켜주는 작업을 한 이후(journal_name_ls)에 이를 바탕으로 함

In [59]:
import pandas as pd
from IPython.display import display
import numpy as np
import re
from collections import Counter
import nltk

In [60]:
df = pd.read_csv('tuned_dataframe.csv')

In [61]:
df.tail()

,paper_name_infor,institution_infor,author_infor,journal_infor,dont_know_infor,page_infor,year_infor,field_infor,cited_infor,char_refer,degree_refer,journal_refer,name_refer,paper_refer,school_refer,year_refer
37242,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[기타자료],NaN,NaN,"Villalonga, Belen",Does Diversification Cause the “Diversificatio...,NaN,1999.0
37243,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Journal of Finance 59 : 479 ~ 506,","Villalonga, Belen",Diversification discount or premium? New Evide...,NaN,2004.0
37244,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Financial Management 33 : 5 ~ 27,","Villalonga, Belen",Does Diversification Cause the Diversification...,NaN,2004.0
37245,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"St. John’s Review 44 : 66 ~ 80,","Weston, J. F",The Nature and Significance of Conglomerate Firms,NaN,1970.0
37246,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,Journal of Finance 56 : 1667 ~ 1691,"Whited, Toni M",Is It Inefficient Investment That Causes the D...,NaN,2001.0


In [62]:
100 * df.isnull().sum() / len(df)

paper_name_infor      0.000000
institution_infor     0.000000
author_infor          0.000000
journal_infor         0.000000
dont_know_infor       0.000000
page_infor            0.000000
year_infor            0.000000
field_infor           0.000000
cited_infor           0.000000
char_refer            0.649717
degree_refer         99.747631
journal_refer        22.570945
name_refer           12.041238
paper_refer           0.649717
school_refer         99.747631
year_refer            0.649717
dtype: float64

# 일단 제목에 Journal 이 있으면 split 해준다.

In [63]:
%%time
for idx,val in enumerate(df['paper_refer']) : 
    if df['paper_refer'].notnull()[idx]:
        if 'Journal' in val : 
            df['journal_refer'][idx] = val[re.search('Journal',val).start():]
            df['paper_refer'][idx] = val[:re.search('Journal',val).start()]

/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


CPU times: user 1min 58s, sys: 19.9 ms, total: 1min 58s
Wall time: 1min 58s


저널 컬럼의 결측치 값이 2퍼센트정도 줄었다.

In [64]:
100 * df.isnull().sum() / len(df)

paper_name_infor      0.000000
institution_infor     0.000000
author_infor          0.000000
journal_infor         0.000000
dont_know_infor       0.000000
page_infor            0.000000
year_infor            0.000000
field_infor           0.000000
cited_infor           0.000000
char_refer            0.649717
degree_refer         99.747631
journal_refer        20.017720
name_refer           12.041238
paper_refer           0.649717
school_refer         99.747631
year_refer            0.649717
dtype: float64

# 다른 저널명들은 기존의 저널 컬럼에서 데이터를 가지고 와 이를 기반으로 찾아준다.

In [65]:
journal_name_ls = list(pd.read_csv('journal_name_ls.csv')['0'].values)

In [66]:
journal_name_ls.append('working paper')

In [67]:
for idx,val in enumerate(journal_name_ls) : 
    if val[-1] == ' ':
        journal_name_ls[idx] = val[:-1]

In [68]:
journal_name_ls = list(set(journal_name_ls))

In [69]:
journal_name_ls[:5]

['Hong Kong Economic Journal Monthly',
 'Emory University',
 'Australasian Accounting Business & Finance Journal',
 'Journal of Forecasting',
 'Pacific-Basin FinanceJournal']

In [70]:
journal_name_ls = [i for i in journal_name_ls if len(i) >2]

In [71]:
%%time
split_ls = []
for idx,val in enumerate(df['paper_refer']):
    if df['paper_refer'].notnull()[idx]:
        for j in journal_name_ls:
            if j in val:
                split_ls.append((idx,j))
                break            

In [72]:
split_ls[:5]

[(5, 'Jan'), (6, 'Infor'), (12, 'Risk'), (13, 'Risk'), (16, 'Risk')]

- 살펴본 결과, 한글일 경우 유의미할 가능성이 높았고, 영어일 경우 Infor , Focus와 같은 단어들은 좋지 않은 Threshold 일 가능성이 높았다.
- 이에 따라서, split_ls 에서 한글은 무조건적으로 집어넣고, 영어인 경우 길이가 10 를 넘어서는 것들만 취합하였다.
- 10으로 숫자를 크게 잡은 이유는 아닌 것을 맞다라고 하는 Type2 error 를 방지하는 것이 우선이라고 생각했기 때문이다.

In [73]:
new_split_ls = [split_ls[i] for i in range(len(split_ls)) if (len(split_ls[i][1]) > 10) or  (re.findall('[가-힣]',split_ls[i][1]))]
new_split_ls[:5]

[(22, 'University of California'),
 (33, 'Corporate Governance'),
 (89, 'Brookings-Wharton Papers on Financial Services'),
 (100, 'Federal Reserve Bank of Chicago'),
 (130, 'Econometrics')]

- 아래의 프로세스는 저널 딕셔너리에서 데이터를 snatch up 해서 저널 컬럼에 집어넣을 때, 기존에 저널이 결측치일 때만 넣어야 함으로, 그 프로세스를 진행하는 것을 의미한다.

In [74]:
idx_ls = df['journal_refer'][df['journal_refer'].isnull()].index
idx_ls = idx_ls.tolist()

In [75]:
len(set([i[0] for i in new_split_ls]).intersection(set(idx_ls)))
nan_idx_ls = sorted(list(set([i[0] for i in new_split_ls]).intersection(set(idx_ls))))
nan_idx_ls[:5]

[1320, 1400, 1810, 2511, 2606]

In [76]:
tuning_ls = []
for idx1,val1 in enumerate(new_split_ls) : 
    if val1[0] in nan_idx_ls : 
        tuning_ls.append(new_split_ls[idx1])
print(len(new_split_ls)) , print(len(nan_idx_ls))
len(tuning_ls) == len(nan_idx_ls)

1205
491


True

In [77]:
tuning_ls[:5]

[(1320, 'Risk Management'),
 (1400, 'Standard and Poor’s'),
 (1810, 'Handbook of the Economics of Finance'),
 (2511, 'Standard and Poor’s'),
 (2606, '보도자료')]

#### 새로운 프로세스

In [78]:
%%time
for idx,val in enumerate(df['paper_refer']) : 
    if idx in [i[0] for i in tuning_ls] :
        #tuning_ls도 일종의 제약식인데, 이 제약식은 journal_refer 가 잘 안잘려져있다. 즉 기존에 없었어야 한다는 제약식이 된다.
        try : 
            loc = [i[0] for i in new_split_ls].index(idx)
            if len(val[re.search(new_split_ls[loc][1],df['paper_refer'][idx]).end() : ]) < 5 :
                df['journal_refer'][idx] = val[re.search(new_split_ls[loc][1],df['paper_refer'][idx]).start() : ]
                df['paper_refer'][idx] = val[ : re.search(new_split_ls[loc][1],df['paper_refer'][idx]).start() ]
        except : print(idx,loc)

/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


CPU times: user 10.7 s, sys: 0 ns, total: 10.7 s
Wall time: 10.7 s


In [79]:
100 * df.isnull().sum() / len(df)

paper_name_infor      0.000000
institution_infor     0.000000
author_infor          0.000000
journal_infor         0.000000
dont_know_infor       0.000000
page_infor            0.000000
year_infor            0.000000
field_infor           0.000000
cited_infor           0.000000
char_refer            0.649717
degree_refer         99.747631
journal_refer        19.660644
name_refer           12.041238
paper_refer           0.649717
school_refer         99.747631
year_refer            0.649717
dtype: float64

In [80]:
len([i for i in df['paper_refer'].tolist() if (type(i) == str) and ( len(i) < 10)])

385

In [81]:
refer_col =[i for i in df.columns if 'refer' in i]

In [83]:
df.loc[[i[0] for i in tuning_ls]][refer_col]

,char_refer,degree_refer,journal_refer,name_refer,paper_refer,school_refer,year_refer
1320,[보고서],NaN,NaN,"Bolder, D. J",Modeling Term Structure Dynamics for Risk Mana...,NaN,2006.0
1400,[기타자료],NaN,NaN,"Servigny, A","Default Correlation: Empirical Evidence, Stand...",NaN,2002.0
1810,[기타자료],NaN,"Handbook of the Economics of Finance,","Barberis, N.","A Survey of Behavioral Finance,",NaN,2003.0
2511,[보고서],NaN,NaN,"Cangemi, B.",Standard and Poor’s Credit Risk Tracker for Pr...,NaN,2003.0
2606,[기타자료],NaN,"보도자료,",한국금융투자협회,"2009년 3/4분기 세계 펀드산업 현황,",NaN,2010.0
3668,[보고서],NaN,NaN,"Dus, I.",Betting on Death and Capital Markets in Retire...,NaN,2003.0
4491,[보고서],NaN,NaN,임홍준,"불확실성을 고려한 벤처기업평가: 사례연구,",NaN,2001.0
5207,[보고서],NaN,NaN,"Bradley, D. J",Venture Capital and IPO Lockup Expiration: An ...,NaN,2000.0
5232,[기타자료],NaN,NaN,금융감독원,"자본시장과 금융투자업에 관한 법률 개정안 입법 예고,",NaN,2011.0
5249,[보고서],NaN,NaN,박정호,"동부증권 애널리스트 보고서,",NaN,2014.0


#### 기존의 프로세스 -- 잘 잘려져 있는 저널임에도 불구하고, 딕셔너리에서 이상한 값들이 새로 채워지는 문제가 발생한다.

In [60]:
%%time
for idx,val in enumerate(df['paper_refer']) : 
    if idx in [i[0] for i in new_split_ls] :
        try : 
            loc = [i[0] for i in new_split_ls].index(idx)
            if not df['journal_refer'][idx]:
                df['journal_refer'][idx] = val[re.search(new_split_ls[loc][1],df['paper_refer'][idx]).start() : ]
                df['paper_refer'][idx] = val[ : re.search(new_split_ls[loc][1],df['paper_refer'][idx]).start() ]
        except : print(idx,loc)

CPU times: user 1.72 s, sys: 3.99 ms, total: 1.73 s
Wall time: 1.73 s


In [63]:
100 * df.isnull().sum() / len(df)

paper_name_infor      0.000000
institution_infor     0.000000
author_infor          0.000000
journal_infor         0.000000
dont_know_infor       0.000000
page_infor            0.000000
year_infor            0.000000
field_infor           0.000000
cited_infor           0.000000
char_refer            0.649717
degree_refer         99.747631
journal_refer        20.017720
name_refer           12.041238
paper_refer           0.649717
school_refer         99.747631
year_refer            0.649717
dtype: float64

- journal refer 에 대한 결측치를 22.57 -> 20.01 -> 18.8 퍼센트로 줄였다.

In [84]:
df.tail()

,paper_name_infor,institution_infor,author_infor,journal_infor,dont_know_infor,page_infor,year_infor,field_infor,cited_infor,char_refer,degree_refer,journal_refer,name_refer,paper_refer,school_refer,year_refer
37242,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[기타자료],NaN,NaN,"Villalonga, Belen",Does Diversification Cause the “Diversificatio...,NaN,1999.0
37243,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Journal of Finance 59 : 479 ~ 506,","Villalonga, Belen",Diversification discount or premium? New Evide...,NaN,2004.0
37244,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Financial Management 33 : 5 ~ 27,","Villalonga, Belen",Does Diversification Cause the Diversification...,NaN,2004.0
37245,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"St. John’s Review 44 : 66 ~ 80,","Weston, J. F",The Nature and Significance of Conglomerate Firms,NaN,1970.0
37246,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,Journal of Finance 56 : 1667 ~ 1691,"Whited, Toni M",Is It Inefficient Investment That Causes the D...,NaN,2001.0


In [85]:
df.to_csv('new_tuned_references.csv',index=False,na_rep='NA')